In [1]:
import torch
from torch import nn
import numpy as np
import src.quantum_object as qo
import src.Operators.operators as op
import src.Qubits.qubits as qu

In [2]:
class BaseQuantumObject(nn.Module):
    def __init__(self, size, num_states, device='cpu'):
        self.size = size
        self.num_states = num_states
        self.device = device
    
    def __define__(self, inp_state):
        if not torch.is_complex(inp_state):
            inp_state = torch.complex(inp_state, torch.zeros_like(inp_state))
        return self.__norm__(inp_state)
        
    def __norm__(self, inp):
        if torch.is_complex(inp):
            norm_const = torch.sqrt((inp*inp.conj()).sum(dim=1).real.unsqueeze(1))
        else:
            norm_const = torch.sqrt((inp*inp).sum(dim=1).real.unsqueeze(1))
        return inp / norm_const

In [3]:
class Qubit(BaseQuantumObject):
    def __init__(self, inp_state, num_states=2, adjoint=False, device = 'cpu'):
        super().__init__(size=1, num_states=num_states, device=device)
        self.num_states = num_states
        self.adjoint = adjoint
        self.device = device
        self.state = self.define(inp_state)
    
    def define(self, inp_state):
        if not isinstance(inp_state, torch.Tensor):
            inp_state = torch.Tensor((inp_state)).reshape((1, self.num_states, 1))
        return super().__define__(inp_state)

In [4]:
class Zero(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=1, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.state = self.define()
    
    def define(self):
        inp_state = torch.Tensor((1, 0)).reshape((1, self.num_states, 1))
        return super().__define__(inp_state)

In [5]:
class One(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=1, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.state = self.define()
    
    def define(self):
        inp_state = torch.Tensor((0, 1)).reshape((1, self.num_states, 1))
        return super().__define__(inp_state)

In [6]:
class Plus(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.state = self.define()
    
    def define(self):
        inp_state = torch.Tensor((1, 1)).reshape((1, self.num_states, 1))
        return super().__define__(inp_state)

In [7]:
class Minus(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.state = self.define()
    
    def define(self):
        inp_state = torch.Tensor((1, -1)).reshape((1, self.num_states, 1))
        return super().__define__(inp_state)

In [8]:
class H(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.gate = self.define()
    
    def define(self):
        gate = torch.Tensor(((1, 1), (1, -1))).reshape((1, 2, 2))
        return super().__define__(gate)

In [9]:
class Gate(BaseQuantumObject):
    def __init__(self, gate, size, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = size
        self.device = device
        self.gate = self.define(gate)
    
    def define(self, gate):
        if not isinstance(gate, torch.Tensor):
            gate = torch.Tensor((gate)).reshape((1, self.num_states**self.size, -1))
        return super().__define__(gate)

In [10]:
class Y(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.gate = self.define()
    
    def define(self):
        gate_re = torch.Tensor(((0, 0), (0, 0))).reshape((1, 2, 2))
        gate_im = torch.Tensor(((0, -1), (1, 0))).reshape((1, 2, 2))
        gate = torch.complex(gate_re, gate_im)
        return super().__define__(gate)

In [11]:
class X(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.gate = self.define()
    
    def define(self):
        gate = torch.Tensor(((0, 1), (1, 0))).reshape((1, 2, 2))
        return super().__define__(gate)

In [12]:
class Z(BaseQuantumObject):
    def __init__(self, device='cpu'):
        super().__init__(size=1, num_states=2, device=device)
        self.num_states = 2
        self.size = 1
        self.device = device
        self.gate = self.define()
    
    def define(self):
        gate = torch.Tensor(((1, 0), (0, -1))).reshape((1, 2, 2))
        return super().__define__(gate)

In [13]:
class CX(BaseQuantumObject):
    def __init__(self, control_qubit, target_qubit, control_state=1, device='cpu'):
        self.control_qubit = control_qubit
        self.target_qubit = target_qubit
        self.control_state = control_state
        self.device = device
        self.size = torch.Tensor([control_qubit, target_qubit]).max() - torch.Tensor([control_qubit, target_qubit]).min()
        self.num_states = 2
        self.bases = self.__bases__()
        self.zero_one = torch.concat((qu.Zero().state, qu.One().state), dim=0)
        self.gate = op.X(self.size).gate
        self.gate_zero_one = torch.concat((torch.matmul(self.gate, self.zero_one[0]), torch.matmul(self.gate, self.zero_one[1])), dim=0)
    
    def __bases__(self):
        bases = torch.arange(self.num_states).reshape((-1, 1)).type(torch.uint8)
        while bases.shape[-1] < self.size:
            _zeros = torch.zeros((bases.shape[0], 1)).type(torch.uint8)
            _ones = torch.ones((bases.shape[0], 1)).type(torch.uint8)
            new_order = torch.concat([_zeros, _ones], dim=0)
            bases = torch.concat((bases, bases), dim=0)
            bases = torch.concat((new_order, bases), dim=1)
        return bases
    
    def __tensor_product__(self, basis):
        kron = basis[0]
        for b in basis[1:]:
            kron = torch.kron(kron, b)
        return kron

    def define(self):
        return self.bases

In [14]:
control_qubit, target_qubit = 1, 5
control_state = 1
cx = CX(control_qubit, target_qubit, control_state=1, device='cpu')
bases = cx.bases
kron = cx.define()
print(bases)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 1, 1, 1],
        [1, 0, 0, 0],
        [1, 0, 0, 1],
        [1, 0, 1, 0],
        [1, 0, 1, 1],
        [1, 1, 0, 0],
        [1, 1, 0, 1],
        [1, 1, 1, 0],
        [1, 1, 1, 1]], dtype=torch.uint8)


In [15]:
print(cx.zero_one.shape)

torch.Size([2, 2, 1])


In [20]:
control_state, control_qubit, target_qubit = 1, 1, 3
cxop = torch.zeros((1, 16, 16))
# _range = max(control_qubit, target_qubit) - min(control_qubit, target_qubit)
# target_qubit = target_qubit - _range
# control_qubit = control_qubit - _range
for i in range(bases.shape[0]):
    b = bases[i]
    in_states = [
        cx.zero_one[b[j].item()] for j in range(b.shape[0])
    ]
    out_states = in_states
    in_states = cx.__tensor_product__(in_states)
    if b[control_qubit].item() == control_state:
        out_states[target_qubit] = cx.gate_zero_one[b[target_qubit].item()]
    out_states = cx.__tensor_product__(out_states)
    in_states = in_states.transpose(0, 1)

RuntimeError: result type ComplexFloat can't be cast to the desired output type Float